In [11]:
import json, numpy
from matplotlib import pyplot as plt

data = {}

method = "fftisdf-60-14"

d1 = None
with open('../data/diamond-krhf-dmet.json', 'r') as f:
    d1 = json.load(f)
d1 = {"/".join(k.split("/")[-2:]): v for k, v in d1.items()}

data = {}
with open('../data/diamond-nk-extrapolate.txt', 'r') as f:
    for line in f:
        if line.strip() == "":
            continue

        line = line.split()
        nk = int(line[2][:-1])
        ene_krhf = float(line[5][:-1])
        ene_sos_mp2 = float(line[8][:-1])
        ene_mp2 = float(line[11][:-1])
        ene_ccsd = float(line[14])

        for kk, vv in d1.items():
            if not kk.split("/")[-1] == "fftisdf-60-14":
                continue
            
            if not vv['nkpt'] == nk:
                continue

            ene_krhf_ref = vv['ene_krhf']
            assert abs(ene_krhf - ene_krhf_ref) < 1e-6

            ene_dmet = vv['ene_dmet']
            kmesh = kk.split("/")[-2]
            natm = vv['natm']

        data[kmesh] = {
            "ene_hf": ene_krhf, "nkpt": nk, "natm": natm,
            "e_corr_sos_mp2": ene_sos_mp2 - ene_krhf,
            "e_corr_lno_mp2": ene_mp2 - ene_krhf,
            "e_corr_lno_ccsd": ene_ccsd - ene_krhf,
            "e_corr_dmet": ene_dmet - ene_krhf,
        }

ene_krhf = d1["10-10-10/" + method]['ene_krhf']
nk = d1["10-10-10/" + method]['nkpt']
ene_dmet = d1["10-10-10/" + method]['ene_dmet']
natm = d1["10-10-10/" + method]['natm']

# data["10-10-10"] = {
#     "ene_hf": ene_krhf, "nkpt": nk, "natm": natm,
#     "e_corr_sos_mp2": numpy.nan,
#     "e_corr_lno_mp2": numpy.nan,
#     "e_corr_lno_ccsd": numpy.nan,
#     "e_corr_dmet": ene_dmet - ene_krhf,
# }

kmesh = ["1-1-2", "1-2-2", "2-2-2", "2-3-3", "3-3-3"]
kmesh += ["3-3-4", "3-4-4", "4-4-4", "4-5-5", "5-5-5"]
kmesh += ["5-5-6", "5-6-6", "6-6-6", "6-7-7", "7-7-7"]
kmesh += ["7-7-8", "7-8-8", "8-8-8", "8-8-10", "8-10-10"]
kmesh += ["10-10-10"]

for km in kmesh:
    vv = data[km]
    out = "kmesh = %10s, nkpt = %4d, natm = %3d, " % (km, vv['nkpt'], vv['natm'])
    out += "ene_hf = % 12.8f, e_corr_dmet = % 12.8f, " % (vv['ene_hf'], vv['e_corr_dmet'])
    out += "e_corr_sos_mp2 = % 12.8f, e_corr_lno_mp2 = % 12.8f, " % (vv['e_corr_sos_mp2'], vv['e_corr_lno_mp2'])
    out += "e_corr_lno_ccsd = % 12.8f" % (vv['e_corr_lno_ccsd'])
    print(out)

data_tdl = {}
method = ["ene_hf", "e_corr_dmet", "e_corr_sos_mp2", "e_corr_lno_mp2", "e_corr_lno_ccsd"]
for m in method:
    xy = []
    for km in kmesh:
        if numpy.isnan(data[km][m]):
            continue
        vv = data[km]
        xy.append((vv["nkpt"], vv[m]))
    x, y = zip(*xy)

    ix = [-1, -2, -3, -4, -5]
    x = 1 / numpy.array(x)[ix]
    y = numpy.array(y)[ix]

    p = numpy.polyfit(x, y, 1)
    data_tdl[m] = p[1]

data["TDL"] = data_tdl


kmesh =      1-1-2, nkpt =    2, natm =   2, ene_hf = -10.57682410, e_corr_dmet =  -0.16123003, e_corr_sos_mp2 =  -0.21644178, e_corr_lno_mp2 =  -0.21762387, e_corr_lno_ccsd =  -0.24557065
kmesh =      1-2-2, nkpt =    4, natm =   2, ene_hf = -10.82926354, e_corr_dmet =  -0.16360076, e_corr_sos_mp2 =  -0.22109749, e_corr_lno_mp2 =  -0.22811055, e_corr_lno_ccsd =  -0.24831476
kmesh =      2-2-2, nkpt =    8, natm =   2, ene_hf = -10.95722433, e_corr_dmet =  -0.16285137, e_corr_sos_mp2 =  -0.22236473, e_corr_lno_mp2 =  -0.23261815, e_corr_lno_ccsd =  -0.25143934
kmesh =      2-3-3, nkpt =   18, natm =   2, ene_hf = -11.02151491, e_corr_dmet =  -0.17574058, e_corr_sos_mp2 =  -0.23188699, e_corr_lno_mp2 =  -0.24966413, e_corr_lno_ccsd =  -0.26200560
kmesh =      3-3-3, nkpt =   27, natm =   2, ene_hf = -11.02227872, e_corr_dmet =  -0.17937400, e_corr_sos_mp2 =  -0.23376449, e_corr_lno_mp2 =  -0.25207692, e_corr_lno_ccsd =  -0.26389312
kmesh =      3-3-4, nkpt =   36, natm =   2, ene_hf = -

In [12]:
# make a latex table
head = """
\\begin{tabular}{cccccc}
\\hline
 & $E_{\\mathrm{HF}}$ & $E_{\\mathrm{DMET}}^{\\mathrm{corr}}$ & $E_{\\mathrm{SOS-MP2}}^{\\mathrm{corr}}$ & $E_{\\mathrm{LNO-CCSD}}^{\\mathrm{corr}}$ \\\\
\\hline"""
print(head)

kmesh = ["2-2-2", "3-3-3", "4-4-4", "5-5-5", "6-6-6", "7-7-7", "8-8-8", "10-10-10"]
kmesh += ["TDL"]

for km in kmesh:
    vv = data[km]
    natm = vv.get("natm", 2)
    if km == "TDL":
        out = "  %25s & " % "TDL"
    else:
        out = "$%25s$ & " % km.replace("-", " \\times ")
    out += "% 12.4f & " % (vv['ene_hf'] / natm)
    out += "% 12.4f & " % (vv['e_corr_dmet'] / natm)
    out += "% 12.4f & " % (vv['e_corr_sos_mp2'] / natm)
    out += "% 12.4f & " % (vv['e_corr_lno_ccsd'] / natm)
    print(out, "\\\\")

print("\\hline")
print("\\end{tabular}")
# print("\\end{table}")



\begin{tabular}{cccccc}
\hline
 & $E_{\mathrm{HF}}$ & $E_{\mathrm{DMET}}^{\mathrm{corr}}$ & $E_{\mathrm{SOS-MP2}}^{\mathrm{corr}}$ & $E_{\mathrm{LNO-CCSD}}^{\mathrm{corr}}$ \\
\hline
$      2 \times 2 \times 2$ &      -5.4786 &      -0.0814 &      -0.1112 &      -0.1257 &  \\
$      3 \times 3 \times 3$ &      -5.5111 &      -0.0897 &      -0.1169 &      -0.1319 &  \\
$      4 \times 4 \times 4$ &      -5.5144 &      -0.0938 &      -0.1196 &      -0.1347 &  \\
$      5 \times 5 \times 5$ &      -5.5142 &      -0.0960 &      -0.1208 &      -0.1359 &  \\
$      6 \times 6 \times 6$ &      -5.5137 &      -0.0975 &      -0.1213 &      -0.1364 &  \\
$      7 \times 7 \times 7$ &      -5.5134 &      -0.0984 &      -0.1216 &      -0.1366 &  \\
$      8 \times 8 \times 8$ &      -5.5131 &      -0.0992 &      -0.1217 &      -0.1368 &  \\
$   10 \times 10 \times 10$ &      -5.5129 &      -0.1002 &      -0.1218 &          nan &  \\
                        TDL &      -5.5127 &      -0.1012 &     